<a href="https://colab.research.google.com/github/ellastra/SocialData_SportsCommunity/blob/f1/Emotion_Analysis_F1_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. 준비 (GPU T4로 세팅)**

In [ ]:
!nvidia-smi -L || echo "No GPU detected (ok for quick tests)"
!pip -q install "transformers" "datasets==3.0.1" "torch==2.3.0+cu118" "torchvision==0.18.0+cu118" "torchaudio==2.3.0+cu118" "sentencepiece" \
                 "pandas" "numpy" "matplotlib" "tqdm" "evaluate" -f https://download.pytorch.org/whl/torch_stable.html

GPU 0: Tesla T4 (UUID: GPU-34c49cce-0a28-a81f-7a51-a8958c3b82c0)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.6/839.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 

**2. Github Link 추가하여 데이터 불러오기, 데이터 파일 명만 추가하여 파일 구분**

In [ ]:
import pandas as pd

# GitHub raw URL 목록
urls = [
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Bahrain_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Australia_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Japan_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Imola_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Spain_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_United_Kingdom_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Hungary_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Netherlands_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Singapore_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Las_Vegas_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Qatar_With_LapNumber.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/f1/CommunityData/Community_Abu_Dhabi_With_LapNumber.csv"
]

dfs = []

for url in urls:
    # 파일 이름만 추출 (예: Round1_Bahrain.csv → Round1_Bahrain)
    filename = url.split("/")[-1].replace(".csv", "")

    # CSV 읽기
    df = pd.read_csv(url)

    # 파일 이름 컬럼 추가
    df["source_file"] = filename

    dfs.append(df)

# 모든 데이터 합치기
df_all = pd.concat(dfs, ignore_index=True)

**3. 데이터 컬럼 선택 및 클리닝**

클리닝
- 디시인사이드의 경우 "dc official App" 삭제
- 해시태그, 멘션, URL, 공백 삭제
- "ㅋㅋㅋ"나 "!!!" 등의 표현 수는 남겨두기

**추후논의하여 클리닝 수정 가능

In [ ]:
import pandas as pd
import numpy as np
import re

# ---------------------------------------------------------------------
# 컬럼 후보 리스트
TEXT_CAND = [
    "text","content","message","body","comment","chat",
    "post_content","post_text","desc","description"
]
TITLE_CAND = ["title","post_title","subject","headline"]
TIME_CAND = [
    "timestamp","time","created_at","createdAt","date","datetime",
    "post_timestamp","post_time","post_date","published_at","created"
]
GROUP_CAND = ["team","team_name","driver","player","user","author","username"]


# ---------------------------------------------------------------------
# 자동 컬럼 탐색 함수
def pick_col(cols, cand_list):
    cols_set = set(cols)
    # 1) 정확히 일치
    for c in cand_list:
        if c in cols_set:
            return c
    # 2) 부분 포함(대소문자 무시)
    for col in cols:
        for k in cand_list:
            if k.lower() in col.lower():
                return col
    return None


def pick_text_columns(cols):
    """본문/텍스트 + 제목 칼럼을 동시에 찾기 위한 유틸"""
    text_col = pick_col(cols, TEXT_CAND)
    title_col = pick_col(cols, TITLE_CAND)
    return text_col, title_col


# ---------------------------------------------------------------------
# "dc official app" 제거 정규식
_DC_OFFICIAL_PAT = re.compile(
    r"""
    ^\s*
    [\-\–\—\u2013\u2014]*\s*
    ["']?\s*
    d\s*c\s*o\s*f\s*f\s*i\s*c\s*i\s*a\s*l\s*a\s*p\s*p
    \s*["']?
    \s*$
    """,
    re.IGNORECASE | re.VERBOSE
)


def remove_dc_official_app_lines(s: str) -> str:
    """멀티라인 텍스트에서 '- dc official app' 류의 단독 라인 제거"""
    lines = re.split(r"[\r\n]+", s)
    kept = [ln for ln in lines if not _DC_OFFICIAL_PAT.match(ln)]
    return "\n".join(kept).strip()


def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""

    # 0) dc official app 제거
    s = remove_dc_official_app_lines(s)

    # 1) URL 제거
    s = re.sub(r"http\S+|www\.\S+", " ", s)

    # 2) 멘션/해시태그 제거
    s = re.sub(r"[@#]\w+", " ", s)

    # 6) 공백 정리
    s = re.sub(r"\s+", " ", s).strip()

    return s


# ---------------------------------------------------------------------
# 데이터 복사
df = df_all.copy()

# 텍스트/제목/시간/그룹 컬럼 자동 선택
TEXT_COL, TITLE_COL = pick_text_columns(df.columns)
TIME_COL = pick_col(df.columns, TIME_CAND)
GROUP_COL = pick_col(df.columns, GROUP_CAND)

print(f"Detected -> TEXT_COL={TEXT_COL}, TITLE_COL={TITLE_COL}, TIME_COL={TIME_COL}, GROUP_COL={GROUP_COL}")

if TEXT_COL is None and TITLE_COL is None:
    raise ValueError("본문/텍스트 컬럼을 찾을 수 없습니다. df.rename(columns={'실제컬럼명':'text'}) 후 재실행하세요.")


# ---------------------------------------------------------------------
# 텍스트 병합 — **NaN/None 방지 버전**
if TEXT_COL is not None and TITLE_COL is not None:
    df["_raw_text_for_clean"] = (
        df[TITLE_COL].fillna("").astype(str) + " " +
        df[TEXT_COL].fillna("").astype(str)
    )
elif TEXT_COL is not None:
    df["_raw_text_for_clean"] = df[TEXT_COL].fillna("").astype(str)
else:
    df["_raw_text_for_clean"] = df[TITLE_COL].fillna("").astype(str)


# ---------------------------------------------------------------------
# 클리닝
df["clean_text"] = df["_raw_text_for_clean"].map(normalize_text)


# ---------------------------------------------------------------------
# 타임스탬프 처리
if TIME_COL is not None:
    if np.issubdtype(df[TIME_COL].dtype, np.number):
        median_val = df[TIME_COL].dropna().astype(float).median() if df[TIME_COL].notna().any() else None

        if median_val and median_val > 1e12:      # ms epoch
            df["ts"] = pd.to_datetime(df[TIME_COL], unit="ms", errors="coerce", utc=True).dt.tz_localize(None)
        elif median_val and median_val > 1e9:     # sec epoch
            df["ts"] = pd.to_datetime(df[TIME_COL], unit="s", errors="coerce", utc=True).dt.tz_localize(None)
        else:
            df["ts"] = pd.to_datetime(df[TIME_COL].fillna("").astype(str), errors="coerce")
    else:
        df["ts"] = pd.to_datetime(df[TIME_COL], errors="coerce")
else:
    df["ts"] = pd.NaT


# ---------------------------------------------------------------------
# 빈 텍스트 제거
df["clean_text"] = df["clean_text"].fillna("").str.strip()
df = df[df["clean_text"].astype(bool)].reset_index(drop=True)


# ---------------------------------------------------------------------
# GP 추출: Community_Bahrain_With_LapNumber → Bahrain
if "source_file" in df.columns:
    m = df["source_file"].str.extract(r"Community_([^_]+)_With_LapNumber", expand=True)
    df["gp_name"] = m[0].fillna("").str.replace("_", " ").str.strip()
    df["round_num"] = np.nan   # round는 이후 매핑으로 추가할 수 있음


# ---------------------------------------------------------------------
print("After cleaning, rows:", len(df))

display_cols = ["clean_text", "ts"]
if "source_file" in df.columns: display_cols.append("source_file")
if "round_num" in df.columns:   display_cols.append("round_num")
if "gp_name" in df.columns:     display_cols.append("gp_name")
if GROUP_COL is not None:       display_cols.append(GROUP_COL)

print(df[display_cols].head(10))

Detected -> TEXT_COL=post_content, TITLE_COL=post_title, TIME_COL=post_timestamp, GROUP_COL=None
After cleaning, rows: 44913
                          clean_text                  ts  \
0                 와 존나설레 ㅅㅂ 가보자고ㅜㅜㅜㅡ 2024-03-03 00:04:04   
1                  스포한다 물탕이가 르끌레 꽂는다 2024-03-03 00:04:06   
2    라마단 <<< 고마우면 개추 ㅋㅋㅋ 덕분에 편하게본다 ㅆ 2024-03-03 00:04:09   
3          와 씹 진짜 저 초록색 흉물 좆같네 ㅅㅂㅋㅋㅋ 2024-03-03 00:04:10   
4                   개ㅈ핀 도착 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 2024-03-03 00:04:07   
5              ㅋㅋㅋ 알핀이 마지막ㅋㅋㅋㅋ ㅋㅋㅋㅋㅋ 2024-03-03 00:04:11   
6  It's lights out and away we go!!! 2024-03-03 00:04:14   
7                          드가좤ㅋㅋㅋㅋㅋㅋ 2024-03-03 00:04:15   
8                          츠노다 가자 ㅋㅋ 2024-03-03 00:04:18   
9                          스타트!!!!!! 2024-03-03 00:04:17   

                        source_file  round_num  gp_name  
0  Community_Bahrain_With_LapNumber        NaN  Bahrain  
1  Community_Bahrain_With_LapNumber        NaN  Bahrain  
2  Community_Bahrain_With_LapNumber     

**4. 파인튜닝을 위한 데이터셋 불러오기**

사용데이터: KOTE, KMRE
- KOTE 한국어 온라인 댓글 감정 라벨링: https://github.com/searle-j/KOTE
- KMRE 한국어 영화 리뷰 댓글 감정 라벨링: https://github.com/passing2961/KMRE

In [ ]:
from datasets import Dataset, DatasetDict
import requests, io, pickle
from datasets import load_dataset

def load_kmre_from_raw(file_url: str):
    resp = requests.get(file_url)
    resp.raise_for_status()
    buf = io.BytesIO(resp.content)
    data = pickle.load(buf)

    # dict 리스트로 변경
    records = []
    for row in data:
        # row가 ["text", "label"] 꼴이라고 가정
        # 혹시 길이가 다르면 건너뛰도록 방어
        if isinstance(row, (list, tuple)) and len(row) >= 2:
            text, label = row[0], row[1]
            records.append({"text": text, "label": label})
        elif isinstance(row, dict):
            # 혹시 이미 dict면 그대로
            records.append(row)

    return Dataset.from_list(records)

base_url = "https://raw.githubusercontent.com/passing2961/KMRE/master/data/"

kmre_train = load_kmre_from_raw(base_url + "kmre_train")
kmre_dev   = load_kmre_from_raw(base_url + "kmre_dev")
kmre_test  = load_kmre_from_raw(base_url + "kmre_test")

kmre = DatasetDict({
    "train": kmre_train,
    "validation": kmre_dev,
    "test": kmre_test,
})

print(kmre)
print(kmre["train"][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 119995
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 29999
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 49997
    })
})
{'text': '너~~~무 재미없어서 중간에 나왔습니다;;', 'label': 2}


**5. KcELECTRA 모델 불러오기**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# KcELECTRA-base (most commonly used)
MODEL_BASE = "beomi/KcELECTRA-base"

# if you're using small / large versions:
# MODEL_BASE = "beomi/KcELECTRA-small-v2022"
# MODEL_BASE = "beomi/KcELECTRA-base-v2022"
# MODEL_BASE = "beomi/KcELECTRA-large-v2022"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_BASE,
    use_fast=True,   # KcELECTRA supports fast tokenizer
)

# num_labels must match your KMRE emotion classes
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_BASE,
    num_labels=6,   # e.g., 6 for anger/disgust/fear/joy/sadness/surprise
)

print("Loaded model:", model.__class__)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded model: <class 'transformers.models.electra.modeling_electra.ElectraForSequenceClassification'>


**6. KoBERT Model Tokenizing & Training(Finetuning)**

In [ ]:
from datasets import DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import numpy as np
import evaluate
import os

# -----------------------------
# 1) 라벨 매핑
# -----------------------------
all_labels = (
    list(set(kmre["train"]["label"]))
    + list(set(kmre["validation"]["label"]))
    + (list(set(kmre["test"]["label"])) if "test" in kmre else [])
)
unique_labels = sorted(list(set(all_labels)))

label2id = {lbl: i for i, lbl in enumerate(unique_labels)}
id2label = {i: str(lbl) for i, lbl in enumerate(unique_labels)}
num_labels = len(unique_labels)

def encode_label(ex):
    ex["labels"] = label2id[ex["label"]]
    return ex

kmre_num = DatasetDict({
    "train": kmre["train"].map(encode_label, remove_columns=["label"]),
    "validation": kmre["validation"].map(encode_label, remove_columns=["label"]),
})
if "test" in kmre:
    kmre_num["test"] = kmre["test"].map(encode_label, remove_columns=["label"])


# -----------------------------
# 2) 토크나이저 / 모델 (KcELECTRA)
# -----------------------------
MODEL_NAME = "beomi/KcELECTRA-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id={str(k): v for k, v in label2id.items()},
)


# -----------------------------
# 3) 토크나이즈
# -----------------------------
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=128,
    )

tokenized_kmre = kmre_num.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"],
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# -----------------------------
# 4) metrics
# -----------------------------
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "f1": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "accuracy": float((preds == labels).mean()),
    }


# -----------------------------
# 5) TrainingArguments
# -----------------------------
OUTPUT_DIR = "./kcelectra_kmre_model"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    weight_decay=0.01,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    gradient_accumulation_steps=1,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)


# -----------------------------
# 6) Trainer
# -----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_kmre["train"],
    eval_dataset=tokenized_kmre["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

print("🚀 Training on GPU (KcELECTRA)...")
trainer.train()
print("✅ Training finished")


# -----------------------------
# 7) 모델 저장
# -----------------------------
SAVE_PATH = OUTPUT_DIR + "/best_model"

os.makedirs(SAVE_PATH, exist_ok=True)

trainer.save_model(SAVE_PATH)               # 모델 가중치 + config 저장
tokenizer.save_pretrained(SAVE_PATH)        # 토크나이저 저장

print("💾 모델 저장 완료! →", SAVE_PATH)


Map:   0%|          | 0/119995 [00:00<?, ? examples/s]

Map:   0%|          | 0/29999 [00:00<?, ? examples/s]

Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/119995 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

/tmp/ipython-input-3965079543.py:114: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Training on GPU (KcELECTRA)...


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,1.146600,1.099339,0.502605,0.581619
2,0.960000,1.027665,0.551983,0.602687
3,0.796500,1.042344,0.559720,0.618787
4,0.662200,1.094200,0.563548,0.621954


✅ Training finished
💾 모델 저장 완료! → ./kcelectra_kmre_model/best_model


모델 결과 저장

In [ ]:
# -----------------------------
# Google Drive
# -----------------------------
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/MyDrive/KcELECTRA_KMRE_Best"
trainer.save_model(DRIVE_PATH)
tokenizer.save_pretrained(DRIVE_PATH)
print("🚀 Google Drive 저장 완료:", DRIVE_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Google Drive 저장 완료: /content/drive/MyDrive/KcELECTRA_KMRE_Best


**7. 감정분석**

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm

# 1) 저장된 모델 경로
LOAD_DIR = "./kcelectra_kmre_model/best_model"

# 2) GPU / CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 3) 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(LOAD_DIR)
model = AutoModelForSequenceClassification.from_pretrained(LOAD_DIR).to(device)
model.eval()

# 4) 배치 유틸
def batched(iterable, batch_size):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]

texts = df["clean_text"].tolist()

all_pred_ids = []
all_pred_labels = []
all_max_probs = []

id2label = model.config.id2label  # {"0":"0","1":"1",...}

batch_size = 64

for batch_texts in tqdm(batched(texts, batch_size), total=(len(texts) + batch_size - 1) // batch_size):
    enc = tokenizer(
        batch_texts,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt",
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        logits = model(**enc).logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()

    pred_ids = probs.argmax(axis=1)
    all_pred_ids.extend(pred_ids)
    all_max_probs.extend(probs.max(axis=1))

    # id2label integer-key or string-key 모두 대응
    for pid in pred_ids:
        label = id2label.get(pid, id2label.get(str(pid), str(pid)))
        all_pred_labels.append(label)

# -------------------------------------------------------
# 5) 먼저 예측 결과를 df에 저장
# -------------------------------------------------------
df["emo_pred_id"] = all_pred_ids
df["emo_pred_label"] = all_pred_labels
df["emo_conf"] = all_max_probs

# -------------------------------------------------------
# 6) 감정 이름 매핑
# -------------------------------------------------------
kmre_label_name = {
    "0": "anger",
    "1": "disgust",
    "2": "fear",
    "3": "happiness",
    "4": "sadness",
    "5": "surprise",
}

df["emo_name"] = df["emo_pred_label"].map(kmre_label_name)

Using device: cuda


  0%|          | 0/702 [00:00<?, ?it/s]

In [ ]:
# 1) CSV로 저장
output_path = "/content/F1_emotion_predictions_kcelectra.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print("💾 Saved to:", output_path)

# 2) Colab에서 바로 다운로드
from google.colab import files
files.download(output_path)

💾 Saved to: /content/F1_emotion_predictions_kcelectra.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>